**Ingesting production table from D365 Finance and Operations and extracting columns we need for M7 analysis.**

In [1]:
from pyspark.sql.functions import *

df = spark.read.format("csv")\
    .option("quote","\"")\
    .option("escape","\"")\
    .option("multiline",True)\
    .option("delimeter",",")\
    .load("Files/Tables/SupplyChain/ProductionControl/WorksheetHeader/ProdTable/PRODTABLE_00001.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Tables/SupplyChain/ProductionControl/WorksheetHeader/ProdTable/PRODTABLE_00001.csv".

StatementMeta(, 5c6dd1db-1efa-4c9b-a7ce-ec478150a0d3, 3, Finished, Available, Finished)

In [2]:
#df = df.sort(df['_c53'].desc())
#display(df)

StatementMeta(, 5c6dd1db-1efa-4c9b-a7ce-ec478150a0d3, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 932192df-6718-4b56-a362-1ddb0b1581d3)

Creating a new dataframe that only contains the columns we need from the production table

In [23]:
renamed_df = df.select(
                col("_c61").alias("ProjectID"),\
                col("_c53").alias("Production"),\
                col("_c6").alias("ItemNumber"),\
                col("_c7").alias("Name"),\
                col("_c15").alias("Quantity"),\
                col("_c85").alias("Created"),\
                col('_c9').alias('Status'),\
                col("_c39").alias("Pool"),\
                col("_c63").alias("ProjectCategory"),\
                col("_c0").alias("ID"),\
                col("_c3").alias("ImportDate"),\
                col('_c52').alias("ReleaseDate"),\
                col('_c41').alias('EstimateDate'))

StatementMeta(, ce17edb1-20a4-4799-af1b-46eacd7a61dc, 25, Finished, Available, Finished)

Sorting by most recent production

In [24]:
renamed_df = renamed_df.sort(renamed_df['Production'].desc())
#display(renamed_df)

StatementMeta(, ce17edb1-20a4-4799-af1b-46eacd7a61dc, 26, Finished, Available, Finished)

Splitting the project id into project and subproject and rebuilding the dataframe

In [25]:
split_col = split(renamed_df['ProjectID'],'-')
renamed_df = renamed_df.withColumn('Proj', split_col.getItem(0))
renamed_df = renamed_df.withColumn('SubProj', split_col.getItem(1))

renamed_df = renamed_df.select(
                "Proj",\
                "SubProj",\
                "Production",\
                "ItemNumber",\
                "Name",\
                "Quantity",\
                "Created",\
                'Status',\
                "Pool",\
                "ProjectCategory",\
                "ID",\
                "ImportDate",\
                'ReleaseDate',\
                'EstimateDate')

#display(renamed_df)

StatementMeta(, ce17edb1-20a4-4799-af1b-46eacd7a61dc, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c935f64d-e6b3-475d-a1cb-274b7ae00424)

Create/overwrite a new table in datalake with the new dataframe

In [26]:
delta_table_path = "Tables/ProdTable" #fill in your delta table path 
renamed_df.write.format("delta").mode("overwrite").option("overwriteSchema","true").save(delta_table_path)

StatementMeta(, ce17edb1-20a4-4799-af1b-46eacd7a61dc, 28, Finished, Available, Finished)